# Notebook 4: Machine Learning Klassifikation mit dem kNN Algorithmus

## Lernziele

* Aus Daten *Features* und *Labels* selektieren.
* Den *knn-Algorithmus* und seine wichtigsten Parameter verstehen.
* Den Algorithmus *trainieren* und mit dem trainierten Algorithmus *Vorhersagen* machen.
* Die Genauigkeit (*Accuracy*) der Vorhersagen des trainierten Algorithmus ermitteln.
* Die Konfusionsmatrix (*Confusion-Matrix*) erstellen und interpretieren.
* Den Sinn des Splits der Daten in *Trainings-Set* und *Test-Set* verstehen und zwischen der *In-Sample* und *Out-of-Sample* Performance unterscheiden.

# Aufgabenstellung und Strukturierung des Problems

Unser Ziel ist es, aus den Längenangaben der Kelch- und Blütenblatter von Iris-Blumen auf die Spezies der Pflanze zu schliessen, d.h. die Spezies zu **prognostizieren**. In Machine Learning Begriffen haben wir es also mit einem **Supervised Learning** Problem zu tun. Die zu prognostizierende Grösse, genannt **Label**, ist hier die Spezies (oder "Klasse") der Blume. Damit haben wir es hier mit einem **Klassifikationsproblem** zu tun. Die **Features** sind die Informationen, mit deren Hilfe wir zu (guten) Prognosen kommen wollen. In unserem Fall sind das die Längenangaben der Blätter.

Erster Schritt eines Machine Learning Prozesses ist es nun, aus dem Datensatz (üblicherweise ein Pandas DataFrame) die Features und Labels zu selektieren.

## Einlesen der Daten

In [ ]:
# Einlesen der Daten-Datei "iris.csv"
import pandas as pd
daten = pd.read_csv('iris.csv')
daten.head()

## Aus den Daten die Features und Labels selektieren

Grundsätzlich kommt jede der vier Längenangaben als einzelnes Feaure in Frage, oder auch alle vier Längenangaben zusammen oder beliebige Kombinationen von zwei oder drei Längenangaben. Es ist oft nicht klar, welche Features am besten geeignet sind. Ein Hilfsmittel, geeignete Features zu finden, sind deskriptive Statistiken und graphische Analysen.

#### Deskriptive Analyse
Wir wollen herausfinden, ob die zur Verfügung stehenden Features *genügend unterschiedlich* sind, um die einzelnen Spezies unterscheiden zu können.

In [ ]:
numeric_columns = ['Sepal_length', 'Sepal_width', 'Petal_length', 'Petal_width']

In [ ]:
# Mittlere Blattlängen der Setosa
daten.loc[daten['class'] == 'Iris-setosa', numeric_columns].mean()

In [ ]:
# Mittlere Blattlängen der Versicolor
daten.loc[daten['class'] == 'Iris-versicolor', numeric_columns].mean()

Die mittleren Blattlängen der drei Spezies unterschieden sich, aber auch *innerhalb einer Spezies* variieren die Längen von Blume zu Blume. Wir müssen also die *Unterschiede in den mittleren Längen der drei Spezies* mit der *Unterschiedlichkeit/Variation der Längen innerhalb der einzelnen Spezies* vergleichen.

In [ ]:
# Mittlere Blattlängen der Setosa
daten.loc[daten['class'] == 'Iris-setosa', numeric_columns].std()

In [ ]:
# Mittlere Blattlängen der Versicolor
daten.loc[daten['class'] == 'Iris-versicolor', numeric_columns].std()

##### Exkurs: Einfacher mit der Pandas *groupby()* Methode: Analyse aller drei Spezies mit einem Befehl

In [ ]:
# Unterschiede in den Mittelwerten der Blattlängen der drei Spezies
daten.groupby('class').mean()

In [ ]:
# Streuung der Längenangaben innerhalb der drei Spezies
daten.groupby('class').std()

Einfacher und übersichtlicher wird es, wenn man die Verteilung der Feature-Werte in geeigneten Diagrammen darstellt.

In [ ]:
# Vorbereitung zum Erstellen von Grafiken
import seaborn as sns  # seaborn Funktionen laden

In [ ]:
# Boxplot in Abhängigkeit der Spezies
sns.boxplot(data = daten, x = 'class', y = 'Sepal_length')

Deutlich bessere Chancen auf eine gute Klassifizierung hat man, wenn man mehr als ein Feature oder alle Featurs verwendet. Die Wirksamkeit von Paaren von Features kann man noch mit Hilfe eines Streudiagramms visualisieren.

In [ ]:
# Streudiagramm mit der "Sepal_length" auf der x-Achse und der "Sepal_width" auf der y-Achse
# Farbgebung der Datenpunkte anhand der Spezies (Feld 'class')
sns.scatterplot(x='Sepal_length', y='Sepal_width', hue='class', data=daten)

### Aufbereiten des Vektors der Labels und der Feature Matrix

Wir sammeln die **Labels** in einem **Vektor y**. Ein Vektor ist ein eindimensionales Objekt, eine "Liste" von Zahlen.

Bemerkung: im Machine Learning wird der **Vektor der Labels** praktisch immer **y** genannt.

In [ ]:
# Die folgende Anweisung liefert als Ergebnis eine 'Pandas-Series'
daten['class']

Eine *Pandas-Series* besteht aus zwei Spalten: die erste Spalte enthält die Zeilennamen (*Indices*) und die zweite Spalte die *Werte*. Damit wir den Vektor der Labels später als Argument in unseren scikit-learn Machine Learning Funktionen benutzen können, müssen wir aus dieser *Pandas-Series* nur die Liste der **Werte** in Form eines *numpy ndarray* extrahieren. Das kann durch anhängen des Attributs *values* erreicht werden:

In [ ]:
y = daten['class'].values  # .values selektiert aus einer Pandas-series die Werte in Form eines numpy ndarrays
y  # y ist nun ein numpy ndarray

In [ ]:
# Zur Kontrolle:
type(y)

In unserem Beispiel wollen wir bei jeder Blume **zwei** Längenangaben als Feature verwenden. Die Features werden immer in einer **Matrix X** gesammelt. Eine Matrix ist ein *zweidimensionales* Objekt. Diese **Feature-Matrix** enthält so viele Spalten wie wir Features haben (in unserem Fall also genau 2) und so viele Zeilen wie wir Daten haben (in unserem Fall also 150, da wir Informationen zu 150 Blumen im Datensatz haben).

Bemerkung: im Machine Learning wird die **Feature-Matrix** praktisch immer **X** genannt.

In [ ]:
# Die folgende Anweisung liefert die Features in Form eines Pandas-DataFrame
daten[['Sepal_length', 'Sepal_width']]

Auch hier benötigen wir für die spätere Weiterverwendung nur die **Werte**, ohne die Zeilen- und Spaltenamen des Pandas DataFrame.

In [ ]:
# Mit dem Attribut .values können wir auch hier die Werte in Form eines numpy ndarrays extrahieren
X = daten[['Sepal_length', 'Sepal_width']].values
X  # X ist nun ein 150*2 numpy ndarray

In [ ]:
# Zur Kontrolle
type(X)

### Etwas *formalere* Zusammenfassung:

Sei $n$ die Anzahl Datenpunkte ('Blumen') im Datensatz, und $m$ die Anzahl der verwendeten Features. Dann ist:
* $\bf{y}$ der **Vektor der Labels**, also ein Vektor mit $n$ Zeilen,
* $y^{(i)}$ der **Label** des $i$-ten Datenpunktes mit $i\in \{1,\ldots,n\}$,
* $\bf{X}$ die $n\times m$ **Feature-Matrix**,
* $x^{(i)} = (x_1^{(i)}, ... , x_m^{(i)})$ der **Feature-Vektor** des $i$-ten Datenpunktes.

### Selektion von Teilen des Vektors der Labels oder der Feature Matrix

Die Selektion von Teilen eines *eindimensionalen* ndarrays haben wir schon im Notebook 2 kennengelernt.

Die Selektion von Zeilen und Spalten eines *zweidimensionalen* ndarrays erfolg ähnlich wie bei Pandas:

array[ZEILENBEREICH, SPALTENBEREICH]

Da Zeilen und Spalten eines ndarrays aber keine Namen haben, wird über den Zeilen- bzw. Spalten-**Index** selektiert. Wie bei Listen beginnt der Index bei 0 für die erste Zeile bzw. Spalte.

In [ ]:
# Beispiel: Feature-Vektor des dritten Datenpunktes (die DRITTE Zeile hat den Index 2!)
X[2]

In [ ]:
# Beispiel: das zweite Feature ALLER Datenpunkte (die ZWEITE Spalte hat den Index 1)
X[:,1]

## Der knn-Machine Learning Algorithmus

Alle (im supervised learning eingesetzten) Machine Learning Algorithmen versuchen auf Basis der Features den Label zu prognostizieren. Der **knn** (*k nearest neighbor*) **Algorithmus** basiert seine Vorhersagen auf der *Mehrheit* derjenigen (bekannten) Daten, deren Features sich *in der Nähe* der Features des zu klassifizierenden Objekts befinden. Das Vorgehen ist in der folgenden Abbildung illustriert. (**ACHTUNG**: in unserem Fall wäre auf den Bildern *X-Axis* = **Sepal_length** und *Y-Axis* = **Sepal_width**.)

![knnBild](KNN_Vorgehen.png)

**Bild links oben**: Gezeigt ist das Streudiagramm der Datenpunkte. Grüne und rote Punkte sind bekannte Datenpunte, d.h. Datenpunkte, deren Features (Koordinaten entlang der x- und y-Achse) und Labels (Farbe der Punkte) bekannt sind. Das Fragezeichen repräsentiert einen Datenpunkt (eine Blume), von dem wir nur die Features (Blattlängen, also die Position im Streudiagramm), nicht aber deren Label (Spezies, Farbe/Klassenzugehörigkeit) kennen. Der kNN Algorithmus soll nun für diese Blume die Spezies prognostizieren.

**Bild rechts oben**: Der kNN Algorithmus versucht Datenpunkte (Blumen) mit ähnlichen Features wie die neue, noch unbekannte Blume zu finden. Dazu muss im Streudiagramm der *Abstand* zu allen bekannten Datenpunkten bestimmt werden.

**Bild unten**: Der Algorithmus berücksichtigt für seine Prognose nur die k=3 *nächstgelegenen* Datenpunkte (Blumen).

Der Klassifikationsentscheid fällt nun auf Basis der **Mehrheit** dieser 3 nächstgelegenen Blumen. In diesem Beispiel würde also für den unbekannten Datenpunkt die Klasse *grün* prognostiziert werden, da zwei der drei nächstgelegenen bekannten Datenpunkte zur Klasse *grün* gehören.

## Wichtige Beobachtungen:

Um die in den obigen Bildern illustrierten Schritte durchführen zu können, müssen wir

* erstens einen **Abstand** berechnen können (wir brauchen eine sogenannte *Metrik*, eine Regel wie Abstände zu messen sind) und
* zweitens sagen, **wie weit** denn die Nachbarschaft reichen soll.

**Lösung:**

**Zu Erstens:**

* Als Metrik wählt man meist die **euklidsche Metrik**: $\rm{Abstand_{2}} = \sqrt{(\Delta x_1)^2 + (\Delta x_2)^2}$, wobei $\Delta x_1$ hier der Abstand der beiden betrachteten Datenpunkte entlang der Achse Sepal_length ist und $\Delta x_2$ der Abstand entlang der Achse Sepal_width. Das ist der Abstand, den wir erhalten würden, wenn wir den Abstand der Punkte auf einem Papier mit dem Lineal abmessen würden. Dies ist die am meisten verwendete Metrik.

* Etwas allgemeiner ist die **Minkowski**-Metrik: $\rm{Abstand_{p}} = \sqrt[p]{(\Delta x_1)^p + (\Delta x_2)^p}$. Wird $p=2$ gesetzt, erhalten wir die oben erwähnte euklidsche Metrik. Für *p=1* erhält man die sogenannte **Manhatten Distanz**: $\rm{Abstand_{1}} = \Delta x_1 + \Delta x_2$. Dies ist die Distanz, die man *in Manhatten* (notabene: rechteckiges Raster von Strassen und Häuserblocks) zurücklegen müsste, um von einer Strassenkreuzung zu einer anderen Strassenkreuzung zu laufen.

* Daneben stehen noch etliche weitere Metriken zur Auswahl, die wir hier aber nicht behandeln.


* **Bemerkung**: Damit die Metrik "vernünftige" Abstände liefert, müssen alle Features (also hier $x_1$ und $x_2$) **ähnliche Grössenordnungen** haben (z.B. wie hier, da beides Längenangaben zwischen etwa 1 und 7 sind). Haben die Features völlig unterschiedliche Grössenordnungen (wenn z.B. $x_1$ der Jahreslohn in CHF und $x_2$ die Anzahl Kinder wäre), so müssen die einzelnen Features zunächst **standardisiert**, d.h. auf eine einheitliche Skala gebracht werden. Diesen Schritt, das sogenannte *Pre-Processing* behandeln wir ggf. später.

* **Weitere Bemerkung**: Wie sollen Abstände bei **kategorialen** Features gemessen werden? Z.B. beim Feature "Geschlecht", was ist der "Abstand" zwischen 'Mann' und 'Frau'? Oder beim Feature "Zivilstand", was ist der Abstand zwischen 'ledig' und 'verheiratet'? Was zwischen 'verheiratet' und 'geschieden'? Und was zwischen 'ledig' und 'geschieden'? Hier gibt es viele Möglichkeiten, solche Kategorien geeignet in numerische Werte oder Vektoren zu transformieren, so dass mit ihnen weiter gerechnet werden kann. Dies ist eine Wissenschaft für sich, die wir hier nicht behandeln können.

**Zu Zweitens:**
* Man wählt die **Gesamtanzahl k** der nächstgelegenen Punkte (*nearest neighbors*), die berücksichtigt werden sollen. Der Algorithmus berechnet also die Abstände (aller) Punkte vom zu Klassifizierenden und berücksichtigt dann bei der Mehrheitsfindung aber nur die *k* Nächstgelegenen. Dieses *k* ist das 'k' in kNN. Um eindeutige Mehrheitentscheide zu erhalten, werden für *k* üblicherweise nur *ungerade* Zahlen gewählt.

### Die kNN-Klasssifikation mit dem Python-Paket scikit-learn

Das Python-Paket scikit-learn (*sklearn*) enthält eine grosse Mengen an Methoden, Funktionen und Algorithmen, die beim Machine Learning benötigt werden. Die vollständige Dokumentation dieses Pakets finden sie unter https://scikit-learn.org/stable/index.html. Wir behandeln hier nur einen winzigen Bruchteil dieses Pakets. Üblicherweise wird beim Programmieren nicht das gesamte Paket importiert (wie wir dies z.B. bei numpy, pandas und seaborn gemacht haben), sondern **nur die benötigten Funktionen/Methoden importiert**.

Syntax dazu: *from sklearn.xxx import yyy*

In [ ]:
# Den knn-Klassifier aus dem scikit-learn importieren
from sklearn.neighbors import KNeighborsClassifier # die kNN-Klassifikations-Methode steht nun zur Verfügung

Im folgenden gehen wir durch die Schritte eines typischen **Machine Learning Prozesses**. Diese Abfolge der Schritte ist fast immer dieselbe, egal welche Algorithmen benutzt werden, ob kNN, Logistische Regression, Entscheidungsbäume oder auch Neuronale Netze.

## 1. Algorithmus festlegen

Zunächst muss der zu verwendende Algorithmus festgelegt werden. Das erfolgt in scikit-learn durch Zuweisung des entsprechenden scikit-learn Objektes zu einer Variablen.

In [ ]:
knn_classifier = KNeighborsClassifier() # Die Variable knn_classifier ist nun ein kNN-Algorithmus

In [ ]:
# Die Variable knn_classifier ist jetztein "KNeighborsClassifier" aus dem scikit-learn Paket
knn_classifier

In [ ]:
knn_classifier.get_params() # zeigt die Parameter des Classifiers und deren Werte.

**Wichtige Parameter** für uns sind:
* *metric* zusammen mit *p*: hier *metric='minkowski'* mit *p=2*, also die *normale* euklidsche Metrik
* *n_neighbors*: hier *n_neighbors=5*, also die 5 nächsten Nachbarn berücksichtigen

Wir belassen beide Parameter hier auf ihren Default Werten.

Interessierte finden die ausführliche Dokumentation zum knn-Classifier unter https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html .

## 2. Den Algorithmus trainieren

Im Lernprozess muss der Algorithmus als nächstes mit den bekannten (Training-)daten trainiert werden. In scikit-learn wird ein Algorithmus trainiert durch Aufruf der Methode *fit()*, der als Argument die Trainingsdaten (Features **und** Labels) mitgegeben werden.

In [ ]:
# Den Klassifier trainieren: Methode .fit()
knn_classifier.fit(X, y)
# ab hier ist der Classifier nun 'trainiert', d.h. er kennt jetzt alle Datenpunkte (X,y) des Samples

**Bemerkung**: Der *Lernprozess* des knn-Algorithmus besteht lediglich daraus, sich die *Daten* (Features und Labels) des zum Trainieren benutzten Samples zu *merken* bzw. sie *auswendig zu lernen*.

## 3. Den Algorithmus anwenden

Mit dem trainierten Algorithmus können nun Voraussagen gemacht werden. In scikti-learn kann man mit der Methode *predict()* Voraussagen machen, der als Argument die Features des zu prognostizierenden Datenpunktes zu übergeben ist. Als Argument erwartet diese Methode **immer** eine **2-dimesionale Feature-Matrix X**. Will man nur ein einzelnes Objekt klassifizieren, besteht die Feature Matrix aus einer einzigen Zeile, ist aber immer noch eine Matrix.

Nehmen wir an wir wollen eine Blume mit Sepal-Länge 5, und Sepal-Breite 3.5 vom Algorithmus klassifizieren lassen. Dann müssen wir der *predict()* Methode eine Feature-Matrix mit einer Zeile und zwei Spalten übergeben.

In [ ]:
# Dieser Aufruf führt zu einem Fehler, da das Argument der predict Methode ein EINDIMENSIONALER Objekt ist
knn_classifier.predict([5, 3.5])

In [ ]:
# So funktionniert es: wir übergeben eine ZWEIDIMENSIONALE Matrix, die aus einer Zeile und zwei Spalten besteht
# Beachten Sie die "doppelten" eckigen Klammern
knn_classifier.predict([ [5, 3.5] ])

Um zu sehen, ob die vom Algorithmus vorhergesagte Klassifikation Sinn macht, schauen wir uns nochmals das Streudiagramm der Trainingsdaten an.

In [ ]:
sns.scatterplot(x = X[:,0], y = X[:,1], hue = y)
# Beachten sie, dass hier der plot-Funktion die Listen der beiden Features (X[:,0] und X[:,1]) als x- und y-Koordinaten übergeben werden.

In [ ]:
# Man kann auch mehrere neue Blumen gleichzeitig klassifizieren lassen: hier eine Matrix mit drei Zeilen (und zwei Spalten)
knn_classifier.predict([[4.5,3],[5.5,3],[7.5,3]])

In [ ]:
# Man kann auch ALLE Blumen des Trainigssets nochmals mit dem Algorithmus auf Basis ihrer Features klassifizierren lassen
knn_classifier.predict(X)

## 4. Performance des Algorithmus messen

In den vorhergehenden Schritten haben wir einen Algorithmus definiert, mit Daten trainiert und dann Vorhersagen machen lassen. Aber sind die Vorhersagen überhaupt gut? Damit kommen wir zum Thema der **Performance Messung**.

### Vorhersagegenauigkeit/Accuracy eines Klassifikationsalgorithmus: scikit-learn Methode *score()*

Ein trainierter Klassifikations-Algorithmus ist grundsätzlich gut, wenn er bei möglichst vielen Daten **korrekte** Vorhersagen macht. Wie können wir herausfinden, ob die Vorhersagen korrekt sind? Dies geht nur mit Daten, bei denen wir **sowohl die Features als auch die Labels kennen**. In unserem Fall ist dies das Sample der 150 bekannten Blumen.

Wir lassen nun den Algorithmus alle 150 Blumen im Sample rein auf Basis ihrer Features klassifizieren und vergleichen die vorhergesagte Spezies mit der tatsächlichen, wahren Spezies der Blume. Der Anteil der korrekt klassifizierten Blumen wird **Accuracy** genannt und liegt immer zwischen 0% und 100%. Da wir die Accuracy auf dem zum **Trainieren** des Algorithmus benutzten Sample auswerten, spricht man von der **In-Sample** Accuracy.

In [ ]:
# Zunächst weisen wir die gemäss Algorithmus vorhergesagte Spezies der 150 Blumen im Sample einer Variablen zu
y_predicted = knn_classifier.predict(X)
y_predicted

In [ ]:
# den Vektor der vorhergesagten Spezies können wir mit dem Vektor der WAHREN Spezies vergleichen (numpy Vektoroperation!!!)
y_predicted == y

In [ ]:
# Wir wollen die Anzahl richtiger Vorhersagen (Anzahl True im obigen Vektor) zählen: numpy Methode sum()
(y_predicted == y).sum()

In [ ]:
# Den Anteil der richtigen Vorhersagen erhalten wir durch Teilen durch die Gesamtanzahl der Vorhersagen.
(y_predicted == y).sum() / len(y)

Mit der scikit-learn Methode *score()* erhält man diese Vorhersagegenauigkeit wesentlich einfacher. Wir müssen dieser Methode lediglich die Feature Matrix und Labels übergeben. Die score-Methode lässt dann zunächst den Algorithmus auf Basis der Feature Matrix die Vorhersagen machen, vergleicht dann mit den ihr übergebenen wahren Labels und gibt als Antwort den Anteil richtiger Vorhersagen zurück.

In [ ]:
# score Methode auf den ganzen Datensatz angewandt. Wir erhalten dasselbe Ergebnis wie oben "von Hand" berechnet.
knn_classifier.score(X, y)

###  Die Konsfusionsmatrix oder Confusion Matrix

Die Confusion Matrix ist ein Hilfsmittel, um zu analysieren, wo bei der Klassifikation Fehler gemacht wurde. Die Confusion-Matrix ist eine Matrix, in deren **Zeilen** die **wahren** (actual) Klassenzugehörigkeiten und in den **Spalten** die vom Algorithmus **vorhergesagten** (Predicted) Klassenzugehörigkeiten gezeigt sind. (Beachten Sie, dass in der Literatur und Wikipedia teilweise die Rolle von Zeilen und Spalten vertauscht ist!).

Hier ein Beipiel einer Konfusionsmatrix:

![confm](ConfusionMatrix.jpg)

Auf der **Diagonalen** der Confusion-Matrix sind **korrekte Vorhersagen** ersichtlich, daneben falsche Vorhersagen. Die obige Matrix besagt z.B., dass 36 Blumen der Spezies Versicolor vom Algorithmus korrekt als Versicolor klassifiziert wurden, jedoch 14 Blumen, die tatsächlich der Spezies Versicolor angehören, fälschlicherweise als Virginica klassifiziert wurden. Entsprechend sehen wir, dass *alle* 50 Blumen der Spezies Setosa korrekt klassifiziert wurden. Bei den Virginica wurden 8 Blumen fälschlicherweise als Versicolor klassifiziert. Daraus gewinnen wir die ggf *wichtige* Erkenntnis, das der Algorithmus sehr erfolgreich die Setosa identifizieren konnte, bei der Unterscheidung zwischen Versicolor und Virginica aber Probleme hatte.

Auch die Vorhersagegenauigkeit (Accuracy) können wir aus der Confusion-Matrix bestimmen: dazu müssen wir die Werte auf der Diagonalen zusammenzählen und durch die Gesamtanzahl der Blumen dividieren. In diesem Beispiel ergäbe sich:

$ Accuracy = (50 + 36 + 42)/(50+36+14+8+42) = 128/150 = 0.853 $

In scikit-learn steht zur Erzeugung der Confusion-Matrix die Funktion *confusion_matrix()* zur Verfügung. Sie hat zwei Argumente: die **wahre** Klassenzugehörigkeit und die vom Algorithmus **vorhergesagte** Klassenzugehörigkeit. Wenn die Klassen wie hier durch Text ('Strings') beschrieben sind, sollte als drittes Argument noch eine Liste der vorkommenden *Labels* angegeben werden. Sind die Klassen durchnummeriert, ist dies nicht nötig. Ausführliche Dokumentation zu dieser Funktion finden Sie unter: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html.

In [ ]:
# Die Funktion aus dem scikit-learn Paket importieren
from sklearn.metrics import confusion_matrix

In [ ]:
# Die Confusion Matrix erzeugen (auf dem gesamten Datensatz)
confusion_matrix(y, knn_classifier.predict(X),
                 labels = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'])
# Durch die dem Argument "labels = " übergebene Liste stellen wir sicher, dass die Reihenfolge der Spezies
# in den Zeilen und Spalten genau die Folgende ist: Setosa, Versicolor, Virginica

In [ ]:
# Berechnung der accuracy aus der Confusion Matrix
(49+38+38)/150

## Split der Daten in *Trainingsset* und *Testset*

Die Performance des Algorithmus auf denselben Daten zu messen, die zum Trainieren benutzt wurden, ist nur die halbe Wahrheit. Natürlich sollte der Algorithmus die zum Trainieren benutzten Daten gut kennen und eine entsprechend gute Performance zeigen. In praktischen Machine Learning Anwendungen interessiert uns aber viel mehr, **wie der Algorithmus auf neuen Daten performt, die er noch nie zuvor gesehen hat**.

Da wir nicht beliebig neue Daten mit bekannten Labels generieren können, ist die Idee, das vorhandene Datensample (auf *zufällige* Art) in **zwei Teile** zu aufzuteilen: ein **Traningsset**, das zum traininieren des Algorithmus benutzt wird, und ein **Testset**, auf dem die Performance des trainierten Algorithmus auf neuen, bisher für den Algorithmus noch unbekannten Daten gemessen werden kann. Mist man die Performance auf dem Testset, spricht man von der **out-of-sample** Performance.

![TTSplit](TrainTestSplit.png)